In [2]:
from numpy import *
from astropy.table import Table
import glob
from astropy.coordinates import SkyCoord
from astropy import units as u
from IPython.display import clear_output
from astropy.table import unique
from astropy.table import vstack

## Get the list of sources
RM_sources = Table.read('y3_gold_2.2.1_wide_sofcol_run2_redmapper_v6.4.22+2_lgt20_vl50_catalog_members.fit')
sources = RM_sources['RA','DEC']
sources['ROW'] = arange(1,len(sources)+1)
sources.add_index('RA') # For future sorting

tolerance = 2.0 #In arcseconds

## For comparing with Cluster Catalog
Verif_Catalog = Table.read('y3_gold_2.2.1_wide_sofcol_run2_redmapper_v6.4.22+2_lgt20_vl50_catalog.fit')
Verif_Catalog = Verif_Catalog['MEM_MATCH_ID','LAMBDA_CHISQ','ILUM'] #If more data is wanted, then more columns will need to be added
Verif_Catalog.add_index('MEM_MATCH_ID') # For future sorting
##

### Table that will be printed out. Can easily be modified to include other data. To do that: do the following
# 1) Add or remove the number of complete brackets in Final_Table
# 2) Assign a column name and assign a type, being float or string or whatever
# 3) Near the end you will see 'Final_Table.add_row'. You'll need to fill in with the data that you want
# ##
Final_Table = Table([[],[],[],[],[],[],[],[],[],[],[],[],[],[]], names=('MEM_MATCH_ID','X-Ray RA','X-Ray DEC','Galaxy RA','Galaxy DEC','P','PFREE','Z','ZLUM','LAMBDA_CHISQ','Galaxy File Row','X-Ray File','X-Ray File Row','Distance'),dtype=('f8', 'f8','f8','f8','f8','f8','f8','f8','f8','f8','i4','U35','i4','U35'))

Locations = glob.glob('Locations/*')
for counter,catalogue in enumerate(Locations,1):
    clear_output()
    print(counter*100.0/len(Locations),'%')
    chips = glob.glob(catalogue+'/*')
#    if counter>50:
#        break
    for chip in chips:

## Tests each file provided against starting list
#         print chip
        try:
            X_sources = Table.read(chip+'/wav_src.fits') # Reads the .fits file
            t = X_sources['COMPONENT','RA','DEC']

            RA = array(t['RA']) # Seperates into coordinates
            DEC = array(t['DEC'])

            RA_max = amax(RA)+tolerance/60.0# Gets max and min values of RA and DEC to search in
            RA_min = amin(RA)-tolerance/60.0
            DEC_max = amax(DEC)+tolerance/60.0
            DEC_min = amin(DEC)-tolerance/60.0

            y = sources.loc['RA',RA_min:RA_max]# Looks for which IDs are in the plate
            y.add_index('DEC')
            z = y.loc['DEC',DEC_min:DEC_max]
            z = z.group_by('ROW')

            #Calculates distance and locates only within tolerance. Does not assume small angle approximation
            X = SkyCoord(ra=t['RA'], dec=t['DEC'])
            catalog = SkyCoord(ra=z['RA']*u.degree, dec=z['DEC']*u.degree)

            idxx, idxcatalog, d2d, d3d = X.search_around_sky(catalog, tolerance*u.arcsecond)
            #Writing table
            d2d_number = 0
            for j,k in zip(idxx,idxcatalog):
                row_RM = z[j-1][2]
                row_X = t[k][0]-1

                radius = str(d2d[d2d_number])

                Final_Table.add_row([RM_sources[row_RM]['MEM_MATCH_ID'],X_sources[row_X]['RA'],X_sources[row_X]['DEC'],RM_sources[row_RM]['RA'],RM_sources[row_RM]['DEC'],RM_sources[row_RM]['P'],RM_sources[row_RM]['PFREE'],RM_sources[row_RM]['Z'],Verif_Catalog.loc['MEM_MATCH_ID',RM_sources[row_RM]['MEM_MATCH_ID']][2],Verif_Catalog.loc['MEM_MATCH_ID',RM_sources[row_RM]['MEM_MATCH_ID']][1],row_RM,chip.replace('Locations/',''),row_X,str(d2d[d2d_number]).replace('0d00m','')])
                d2d_number+=1
        except:
            pass
#print Final_Table #For Debugging purposes
# Final_Table.write('table_Y3.csv', format='ascii.csv',overwrite='true')# To have full match list
print('Finished Finding AGN')

## Editing table to include unique matches only. Can be ignored if accounted for later
qw = Final_Table
t = unique(qw, keys=["MEM_MATCH_ID",'Galaxy RA','X-Ray DEC'])
t.write('table_Y3_unique.csv',overwrite=True) #Final match list


## Table for catalog members are in clusters with at least one AGN source
t = unique(t, keys="MEM_MATCH_ID")
ending = Table()
for i in t['MEM_MATCH_ID']:
    
    new = RM_sources[RM_sources['MEM_MATCH_ID']==i]
    ending = vstack([new,ending])

ending.write('catalog_members_relevant.fits',overwrite=True)


FileNotFoundError: [Errno 2] No such file or directory: 'y3_gold_2.2.1_wide_sofcol_run2_redmapper_v6.4.22+2_lgt20_vl50_catalog_members.fit'